# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [90]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [91]:
import import_ipynb


import warnings
warnings.filterwarnings('ignore')

from challenge_1 import *



From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [92]:
# your code here

pokemon["total_power"] = pokemon["HP"]+pokemon["Attack"]+pokemon["Defense"]+pokemon["Sp. Atk"]+pokemon["Sp. Def"]+pokemon["Speed"]

pokemon["total_coincidence"] = pokemon["Total"] == pokemon["total_power"]

coincidentes = [element for element in pokemon["Name"] if pokemon.loc[pokemon["Name"]==element, "total_coincidence"].item() == False]

coincidentes
             



[]

In [93]:
display(pokemon.head())

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,A/D Ratio,Combo Type,total_power,total_coincidence
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,1.000000,Grass-Poison,318,True
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,0.984127,Grass-Poison,405,True
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,0.987952,Grass-Poison,525,True
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,0.813008,Grass-Poison,625,True
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False,1.076923,Fire-Flying,534,True


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [141]:
# your code here

onehot_1 = pd.get_dummies(pokemon["Type 1"])

onehot_2 = pd.get_dummies(pokemon["Type 2"])

onehot = onehot_1 + onehot_2

onehot = onehot.fillna(0)

pokemon = pd.concat([pokemon, onehot], axis=1)

duplicadas = pokemon.T.duplicated()

pokemon = pokemon.loc[:, ~duplicadas]

display(onehot)


,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
796,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
797,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
798,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [147]:
# your code here

lista_corr = ["Total"]

for e in pokemon["Type 1"]:
    if e not in lista_corr:
        lista_corr.append(e)

for e in pokemon["Type 2"]:
    if e not in lista_corr:
        lista_corr.append(e)


correlacion = pokemon[lista_corr].corr()

print(f"El tipo de Pokemon que mas fuerte pega es {max(correlacion)}")

display(correlacion)

El tipo de Pokemon que mas fuerte pega es Water


,Total,Grass,Fire,Bug,Normal,Poison,Water,Rock,Electric,Ghost,Ground,Psychic,Ice,Dragon,Fairy,Dark,Steel,Fighting,Flying
Total,1.000000,-0.108008,0.135961,-0.159988,-0.175574,-0.157293,-0.051025,-0.000605,0.050108,-0.045354,-0.014493,0.093013,0.046567,0.324208,-0.071636,0.028288,0.083201,0.143950,-0.024826
Grass,-0.108008,1.000000,-0.129518,-0.044608,-0.093820,0.169878,-0.129277,-0.111861,-0.072237,0.110717,-0.142444,-0.097787,-0.021142,-0.112172,-0.049559,-0.071159,-0.100800,-0.048537,-0.155934
Fire,0.135961,-0.129518,1.000000,-0.070279,-0.044919,-0.110439,-0.112330,-0.086535,-0.037424,-0.003968,-0.017707,-0.039363,-0.078235,-0.040830,-0.078235,-0.016221,-0.078607,0.130731,-0.052427
Bug,-0.159988,-0.044608,-0.070279,1.000000,-0.129769,0.151559,-0.152669,-0.033259,-0.032794,-0.095536,-0.109331,-0.148348,-0.099227,-0.126227,-0.099227,-0.129769,0.026661,-0.062643,0.014145
Normal,-0.175574,-0.093820,-0.044919,-0.129769,1.000000,-0.118646,-0.123728,-0.121476,-0.009806,-0.102316,-0.104395,-0.076856,-0.084049,-0.106919,0.085689,-0.109920,-0.114331,-0.037861,0.269114
Poison,-0.157293,0.169878,-0.110439,0.151559,-0.118646,1.000000,-0.074555,-0.131120,-0.086794,-0.002350,-0.093383,-0.135632,-0.090722,-0.089341,-0.090722,-0.042176,-0.123407,-0.049095,-0.147071
Water,-0.051025,-0.129277,-0.112330,-0.152669,-0.123728,-0.074555,1.000000,0.042028,-0.020678,-0.089055,0.024554,-0.067591,0.053802,-0.096803,-0.056330,-0.013948,-0.130250,-0.056676,-0.138712
Rock,-0.000605,-0.111861,-0.086535,-0.033259,-0.121476,-0.131120,0.042028,1.000000,-0.088864,-0.113072,0.057920,-0.093741,-0.030102,-0.066963,0.001289,-0.071410,0.019222,-0.080224,-0.135286
Electric,0.050108,-0.072237,-0.037424,-0.032794,-0.009806,-0.086794,-0.020678,-0.088864,1.000000,-0.037424,-0.062622,-0.091923,-0.017216,-0.006017,-0.017216,-0.080411,0.019009,-0.071379,-0.012361
Ghost,-0.045354,0.110717,-0.003968,-0.095536,-0.102316,-0.002350,-0.089055,-0.113072,-0.037424,1.000000,-0.068615,-0.091097,-0.042247,-0.040830,-0.078235,-0.016221,0.004837,-0.090824,-0.132815


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [96]:
# your code here